In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [33]:
# CUDA, 실행 device setting
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [53]:
EPOCHS = 50
BATCH_SIZE = 64

In [54]:
# data augmentation
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307, ), (0.3081, ))
                   ])
                  ),
    batch_size = BATCH_SIZE,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307, ), (0.3081, ))
                          ])
                         ),
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [55]:
# ANN settings + dropout
# dropout, 학습 시 일부 tensor만 활용 overfitting을 막기 위해 사용한다.
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        # dropout
        self.dropout_p = dropout_p 
        
    def forward(self, x):
        # 1차원 행렬로 변환
        x = x.view(-1, 784)
        # 각 레이어를 거쳐 활성화 함수 실행, 다음 뉴런으로 진행
        x = F.relu(self.fc1(x))
        # dropout
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = F.relu(self.fc2(x))
        # dropout
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = self.fc3(x)
        return x

In [56]:
# 설정한 실행 환경에 모델 선언
# dropout할 비율 설정, 전체 tensor 중 20%를 사용하지 않는다.
model = Net(dropout_p=0.2).to(DEVICE)

# 최적화 알고리즘 설정
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [57]:
# 학습 함수
def train(model, train_loader, optimizer):
    model.train()
    
    for batch_dix, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [58]:
# 평가 함수 
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0 # 예측을 맞춘 수를 저장하기 위한 변수
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            # 가장 큰 값을 가진 클래스가 모델의 예측
            # 예측과 정답을 비교하여 일치할 경우 correct에 1을 더한다.
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        # 정확도 측정
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [59]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.6546, Accuracy: 77.29%
[2] Test Loss: 0.5338, Accuracy: 80.96%
[3] Test Loss: 0.4905, Accuracy: 82.32%
[4] Test Loss: 0.4639, Accuracy: 83.10%
[5] Test Loss: 0.4425, Accuracy: 83.81%
[6] Test Loss: 0.4304, Accuracy: 84.60%
[7] Test Loss: 0.4161, Accuracy: 84.96%
[8] Test Loss: 0.4162, Accuracy: 84.76%
[9] Test Loss: 0.3995, Accuracy: 85.61%
[10] Test Loss: 0.3966, Accuracy: 85.43%
[11] Test Loss: 0.3855, Accuracy: 85.98%
[12] Test Loss: 0.3883, Accuracy: 85.92%
[13] Test Loss: 0.3804, Accuracy: 86.30%
[14] Test Loss: 0.3687, Accuracy: 86.65%
[15] Test Loss: 0.3694, Accuracy: 86.63%
[16] Test Loss: 0.3720, Accuracy: 86.36%
[17] Test Loss: 0.3618, Accuracy: 86.88%
[18] Test Loss: 0.3578, Accuracy: 86.96%
[19] Test Loss: 0.3638, Accuracy: 86.82%
[20] Test Loss: 0.3527, Accuracy: 87.49%
[21] Test Loss: 0.3557, Accuracy: 87.15%
[22] Test Loss: 0.3525, Accuracy: 86.91%
[23] Test Loss: 0.3463, Accuracy: 87.52%
[24] Test Loss: 0.3431, Accuracy: 87.65%
[25] Test Loss: 0.3523, A